In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
df = pd.read_csv('raw_customer_sales_data.csv')
df.head(1)

In [ ]:
# Getting overall idea of, What we have in our df.
df.info()

In [ ]:
# Getting a statistical idea of Numeric columns in df
df.describe()

In [ ]:
# Finding null value counts for each column
df.isna().sum()

# Handling NaN values in Numerical Columns

In [ ]:
# df.loc[df['CustomerID'].isna()]

# Deleting records with no CustomerID because these records doesn't matter.
df = df.dropna(subset=['CustomerID'], ignore_index=True)



# Cleaning Age column standardizing and filling NaN values.

# Creating extract_age function.
def extract_age(age):
    age_num = re.findall('[0-9]+', str(age))
        return age_num[0]
    else:
        return age

# Extracting age from Age col.
df['Age'] = df['Age'].apply(lambda x: extract_age(x))

In [ ]:
# Filling null age with Age col median.
df['Age'] = df['Age'].fillna(int(df['Age']
                                 .dropna()
                                 .astype('int64')
                                 .median()))
df['Age'].unique()

In [ ]:
# Filling PurchaseAmount NaN values with median of PurchaseAmount because of outliers.
df['PurchaseAmount'] = df['PurchaseAmount'].fillna(df['PurchaseAmount'].median())

In [ ]:
# Filling FeedbackScore NaN values with mode of FeedbackScore because it's categorical.
df['FeedbackScore'] = df['FeedbackScore'].fillna(df['FeedbackScore'].mode()[0])

# Handling inconsistent values in Categorical columns

In [ ]:
# Finding inconsistent values amongst categorical columns.
for col in df.columns:
    if df[col].nunique() < 20:
        print(df[col].value_counts())
        print('-'*50)

In [ ]:
# Changing format of values in Gender , City and Country with .title()
for col in ['Gender','City','Country']:
    df[col] = df[col].str.title().str.strip()

In [ ]:
# Changing values in Gender and Country columns.
df['Gender'] = df['Gender'].replace({'M': 'Male', 'F':'Female'})
df['Country'] = df['Country'].replace({'Usa': 'USA', 'Uk':'UK'})

# Handling NaN values in Categorical Columns

In [ ]:
df.columns

In [ ]:
# Filling NaN in Gender and City with 'Unknown'
df['Gender'] = df['Gender'].fillna('Unknown')
df['City'] =df['City'].fillna('Unknown')

In [ ]:
# Finding records where city name is present but not the country name.
x = df.loc[(df['City'] != 'Unknown') & (df['Country'].isnull()), ['City','Country']]
# From the records we got earlier finding unique city names.
x['City'].unique()

In [ ]:
# Since All the city names are from India We can phil these records country name with India
# And rest with unknown.
city_to_country = {'Chennai':'India',
                   'Hyderabad':'India',
                   'Kolkata':'India',
                   'Mumbai':'India',
                   'Delhi':'India',
                   'Bangalore':'India'}
# Creating a series using map() and using it 
df['Country'] = df['Country'].fillna(df['City'].map(city_to_country))
df['Country'] =df['Country'].fillna('Unknown')

# Handling Duplicates

In [ ]:
# Deleting duplicate records on the basis of CustomerID
# We are using primary key logic here.
df = df.drop_duplicates(subset=['CustomerID'], ignore_index=True)

In [ ]:
df['CustomerID'].nunique()  # No dupes present

# Normalizing column names and changing data types

In [ ]:
df = df.rename(columns={'CustomerID': 'Customer_ID',
                        'SignupDate': 'Signup_Date',
                        'LastPurchaseDate': 'Last_Purchase_Date',
                        'PurchaseAmount': 'Purchase_Amount',
                        'FeedbackScore': 'Feedback_Score',
                        'PhoneNumber': 'Phone_Number'})

In [ ]:
# Converting categorical columns into category data types helps 
# reducing storage size of the file.
df = df.astype({'Gender': 'category',
                'City': 'category',
                'Country': 'category'})

In [ ]:
# Age column have mixture strings and integer so first we force age into int
# And then we convert it into Int64.
df['Age'] = pd.to_numeric(df['Age'], errors='coerce').astype('Int64')

In [ ]:
# Converting Signup_Date and Last_Purchase_Date Data types in date time.
df['Signup_Date'] = pd.to_datetime(df['Signup_Date'])
df['Last_Purchase_Date'] = pd.to_datetime(df['Last_Purchase_Date'])

# Checking cleaned data readiness for analysis

In [ ]:
# Making copy of df and creating a new feature from Age column

categorized = df.copy()
categorized['Age_Category'] = categorized['Age'].apply(lambda x: 'Elder' if x >= 60 else 
                                                                 ('Adult' if x >= 30 else 
                                                                 ('Young' if x >= 20 else 'Teen') ))

In [ ]:
categorized[['Age','Age_Category']].head(10)

In [ ]:
import matplotlib.pyplot as plt

# Aggregate total purchase by age category
purchase_by_age = (categorized.groupby('Age_Category')
                              .agg(Purchase_Amount_Millions=('Purchase_Amount', 'sum'))
                              .reset_index()
                  )
purchase_by_age['Purchase_Amount_Millions'] = purchase_by_age['Purchase_Amount_Millions'] / 1000000

# Plotting horizontal bar chart
plt.style.use('dark_background')
plt.barh(y=purchase_by_age['Age_Category'], width=purchase_by_age['Purchase_Amount_Millions'])
plt.xlabel('Total Purchase Amount (Millions)')
plt.ylabel('Age Category')
plt.title('Total Purchases by Age Category')
plt.show()


In [ ]:
# import pandas as pd
# import numpy as np
# from faker import Faker
# import random

# fake = Faker()

# # Number of records
# n = 10200

# # Helper functions for injecting errors
# def random_gender():
#     return random.choice(["M", "F", "m", "f", "male", "female", "FEMALE", None])

# def random_age():
#     choice = random.choice([
#         str(random.randint(18, 70)),                     # normal int
#         str(float(random.randint(18, 70))),              # float
#         f"{random.randint(18, 70)} years",               # with text
#         str(random.randint(-10, -1)),                    # negative values
#         None                                             # missing
#     ])
#     return choice

# def random_city():
#     cities = ["Delhi", "Mumbai", "Kolkata", "Hyderabad", "Chennai", "Bangalore"]
#     city = random.choice(cities)
#     # Random casing
#     return random.choice([city.lower(), city.upper(), city.capitalize(), None])

# def random_country():
#     countries = ["India", "USA", "Canada", "UK", "Australia"]
#     country = random.choice(countries)
#     # Random casing and occasional missing
#     return random.choice([country.lower(), country.upper(), country.capitalize(), "InDia", None])

# def random_purchase_amount():
#     # Normal purchase amounts
#     amount = round(random.uniform(1000, 50000), 2)
#     # Inject outliers
#     if random.random() < 0.01:  # 1% chance
#         amount = random.choice([999999.99, 0.01, 1000000])
#     # Inject missing
#     if random.random() < 0.05:
#         return None
#     return amount

# def random_feedback():
#     if random.random() < 0.1:
#         return None
#     return random.randint(1, 10)

# def random_customer_id(i):
#     if random.random() < 0.05:  # 5% missing
#         return None
#     return f"CUST{i}"

# def random_last_purchase_date():
#     if random.random() < 0.1:  # 10% missing
#         return None
#     return fake.date_between(start_date="-5y", end_date="today").isoformat()

# # Generate dataset
# data = {
#     "CustomerID": [random_customer_id(i) for i in range(1, n+1)],
#     "Name": [fake.name() for _ in range(n)],
#     "Gender": [random_gender() for _ in range(n)],
#     "Age": [random_age() for _ in range(n)],
#     "City": [random_city() for _ in range(n)],
#     "SignupDate": [fake.date_between(start_date="-10y", end_date="today").isoformat() for _ in range(n)],
#     "LastPurchaseDate": [random_last_purchase_date() for _ in range(n)],
#     "PurchaseAmount": [random_purchase_amount() for _ in range(n)],
#     "FeedbackScore": [random_feedback() for _ in range(n)],
#     "Email": [fake.email() for _ in range(n)],
#     "PhoneNumber": [fake.msisdn()[:10] for _ in range(n)],  # always filled
#     "Country": [random_country() for _ in range(n)]
# }

# df = pd.DataFrame(data)

# # Preview
# print(df.head())
# print(df.info())
# df.to_csv('raw_customer_sales_data.csv', index=False)